In [28]:
sc.version
val conf = sc.hadoopConfiguration
val fs = org.apache.hadoop.fs.FileSystem.get(conf)
val hdfshome = fs.getHomeDirectory
println(sc.version)
println(hdfshome)
val files = fs.listFiles(hdfshome, false)

2.2.0
hdfs://spark-clu-m/user/root


In [56]:
val rootDir = "/user/fsainz/DATA/in"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)

In [57]:
files.next

LocatedFileStatus{path=hdfs://spark-clu-m/user/fsainz/DATA/in/a01.dat; isDirectory=false; length=5914000; replication=2; blocksize=134217728; modification_time=1511166838942; access_time=1511166838450; owner=fsainz; group=hadoop; permission=rw-r--r--; isSymlink=false}

In [61]:
val rootDir = "hdfs://spark-clu-m/user/fsainz/DATA/in"
val files = fs.listFiles(new org.apache.hadoop.fs.Path(rootDir), false)
files.next

LocatedFileStatus{path=hdfs://spark-clu-m/user/fsainz/DATA/in/a01.dat; isDirectory=false; length=5914000; replication=2; blocksize=134217728; modification_time=1511166838942; access_time=1511166838450; owner=fsainz; group=hadoop; permission=rw-r--r--; isSymlink=false}

In [67]:
fs.exists(new org.apache.hadoop.fs.Path("hdfs://spark-clu-m/user/fsainz/DATA/in/a02.dat"))


true

## Split in windows


In [76]:
val rootDir = "/user/fsainz/DATA/in/"
val inFile = rootDir + "a02.dat"


val rawData = sc.textFile(inFile)

In [82]:
val n = rawData.count
val first = rawData.first()

In [89]:
val lens = rawData.map(x => (x.length))
lens.reduce( _+ _)

6309860

In [95]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

val WINDOW = 32;
val STEP = 2;
val SAMPLES = 200000;
val scale = 1/200.0;


// TODO:
//How to read a binary file from spark, that is partioned...

// Till Now start reading local file
val fileName = "/home/fsainz/src/data/a02.dat"
var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)


val nSamples =  (inFile.length() / 2)   // We are going to read short 2 bytes..
val nWindows = nSamples / WINDOW
println("nWindows = " + nWindows.toString)

val buffer = new Array[Double](WINDOW)
var windows = scala.collection.mutable.ListBuffer.empty[Array[Double]]

var pos = 0
var available = 0
var dataWindow = new Array[Double](WINDOW)

for (  w  <- 1 to nWindows.toInt){
    // new object is neeed to append at the end!!
  dataWindow = new Array[Double](WINDOW) 
  var newWin:Boolean = false
  while( ! newWin) {
    val value = input.readShort()
    buffer( pos ) = (value.toDouble * scale) 
    available += 1
    pos +=1
    if ( pos == WINDOW )  // buffer ends
      pos = 0  
    if (available == WINDOW) {    
      var j =0
      for( i <- pos until WINDOW){
         dataWindow(j) = buffer(i)
         j +=1
      }
      for( i <- 0 until pos ){
         dataWindow(j) = buffer(i)
         j +=1
      }
      //dataWindow.foreach(print)
      //println("|")
      newWin = true
      available = WINDOW-STEP   // ready for read windows displacement offset 
    }
  }
  windows.append(dataWindow  )  
}
input.close()

nWindows = 99437


In [96]:
nWindows*32*2

6363968

## Defining windowing function

In [99]:

var windowingF  =  new Array[Double](WINDOW)   //differs of Array[Double](WINDOW) uajj
for ( i <- 0 until WINDOW) {
    val y = Math.sin(Math.PI * i / (WINDOW - 1.0));
          windowingF(i)=  y * y
 }


In [107]:
def vectorByvector( a : Array[Double], b:Array[Double]) : Array[Double]  = {
  val l = a.length
  assert( (b.length == l) )
    
  val ret = new Array[Double](l)
  for ( i <- 0 until l)
    ret (i) = a(i) * b(i) 
  ret
}

In [108]:
// spark 2.1 mllib
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.regression.LabeledPoint 

In [115]:
val ECGRdd = sc.parallelize(windows)
val ECGwnRdd = ECGRdd.map( x => vectorByvector( x,windowingF)).map(x =>( x, Vectors.norm( Vectors.dense(x),2) )).map(x => Vectors.dense(x._1.map( c => c/ x._2)) ) 
ECGwnRdd.count

99437

In [116]:
    ECGwnRdd.cache()  // important to kmeans time
    val numIterations = 20// we want to assure a global mininum with 20 it seem not to be reached
    val numClusters = 400
    val clusters = KMeans.train(ECGwnRdd, numClusters, numIterations)

99437


In [117]:
clusters

org.apache.spark.mllib.clustering.KMeansModel@7afb808

In [119]:
clusters.save(sc, "/user/fsainz/DATA/out/clusters.model")